In [49]:
import requests
import io

import numpy as np
import pandas as pd

from pycoingecko import CoinGeckoAPI

from bs4 import BeautifulSoup as bts

from selenium import webdriver

from PyPDF2 import PdfReader

### Notes

**Why isn't this a script?**

We are only generating the dataset once. We don't need to write a script that does that is properly formatted and tested, we can do it using a notebooks and test our results at the end.

**Wtf is going on here?**

Haha, yeah it's kind of complicated. And very possibly (probably) overly engineered and convoluted. Oh well.

This what's going on:

1. Use the CG API to extract market caps of top 100 coins
2. Scrape the ramining market caps of all other coins from CG web
3. Get whitepaper pdf page urls by scraping whitepaper.io
4. Use those to obtain the raw pdf urls
5. Feed those pdf urls into a PDF parser and extract pdf data

**Lots of data is wrong!!!**

Some of the data might be slightly off but it is all roughly correct. We only need to be roughly correct here because 1) we're primarily concerned with relativity (features systematically reduced due to bad data extraxction doesn't matter) 2) errors will be small (read code) and 3) I don't really care because this whole thing is just kind of illustrative.

## Market Cap DataFrame

### Top 100

First we use the CG API to get the first 100 (by market cap).

In [3]:
# Make a df with all api coin data
cg = CoinGeckoAPI()
api_cap_df = pd.DataFrame(cg.get_coins_markets(vs_currency="usd"))

In [4]:
api_cap_df.head(3)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,61100.000000,1206547874874,1,1.283670e+12,14427173721,61403.000,...,2.100000e+07,21000000.0,73738.00,-17.07426,2024-03-14T07:10:36.635Z,67.810000,90076.41862,2013-07-06T00:00:00.000Z,None,2024-08-11T07:30:53.636Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,2663.870000,320467809737,2,3.204678e+11,8506188310,2667.870,...,1.202678e+08,NaN,4878.26,-45.35849,2021-11-10T14:24:19.604Z,0.432979,615531.67996,2015-10-20T00:00:00.000Z,"{'times': 57.276469847229016, 'currency': 'btc...",2024-08-11T07:30:53.784Z
2,tether,usdt,Tether,https://coin-images.coingecko.com/coins/images...,0.999907,115641341009,3,1.156413e+11,23818875228,1.002,...,1.156386e+11,NaN,1.32,-24.42957,2018-07-24T00:00:00.000Z,0.572521,74.64317,2015-03-02T00:00:00.000Z,None,2024-08-11T07:30:55.868Z


### All the rest

Unfortunately the API (free version) only allows us to access the top 100 coins.

For the others, we scrape their market cap and add this scraped data to a df.

In [5]:
scrape_dict_list = []  # instantiate empty list
for page_num in range(2, 47):  # only have market cap info for coins up to page 47 on CG
    # Get HTML for the page
    url = 'https://www.coingecko.com/?page='+ str(page_num)
    result = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    # Get list of names and tickers
    raw_name_list = soup.findAll('div', class_='tw-text-gray-700 dark:tw-text-moon-100 tw-font-semibold tw-text-sm tw-leading-5')
    name_ticker_list = [raw_name.text.split() for raw_name in raw_name_list][1:-1] # [1:] to skip "Highlights"
    # Get list of market caps
    if page_num == 2:
        start_mc = 800000000  # starting market cap for the second page of results
    else:
        start_mc = scrape_dict_list[-1]['market_cap']
    raw_price_list = soup.findAll('span', {'data-price-target': 'price'})
    mc_list = [start_mc]
    for i in range(10, len(raw_price_list)):
        num_item = float(raw_price_list[i].text.replace('$', '').replace(',', ''))
        if mc_list[-1] > num_item > mc_list[-1]*0.8:  # to ensure we're finding market cap and not other values
            mc_list.append(num_item)
    mc_list = mc_list[1:] # [1:] to skip start_mc
    # Combine into a dictionary and append to list
    if len(name_ticker_list) <= len(mc_list):
        scrape_len = len(name_ticker_list)
    else:
        scrape_len = len(mc_list)
    for i in range(scrape_len):
        coin_dict={}
        coin_dict['symbol'] = name_ticker_list[i][-1]
        coin_dict['name'] = ' '.join(name_ticker_list[i][:-1])
        coin_dict['market_cap'] = mc_list[i]
        scrape_dict_list.append(coin_dict)

In [6]:
# Make a dataframe with all scrape coin data
scrape_cap_df = pd.DataFrame(scrape_dict_list)

In [7]:
scrape_cap_df.head(3)

,symbol,name,market_cap
0,GALA,GALA,653478314.0
1,ENS,Ethereum Name Service,648100282.0
2,FRAX,Frax,647605253.0


### Combine to one market cap df

In [8]:
# Transform API df
api_trans_df = api_cap_df[['name', 'market_cap']]
# Transform scrape df
scrape_trans_df = scrape_cap_df[['name', 'market_cap']].astype({'market_cap': 'int64'})

In [9]:
# Union - should be roughly disjoint sets
combined_cap_df = pd.concat([api_trans_df, scrape_trans_df], axis=0)
# Dropping duplicates to be safe
clean_combined_cap_df = combined_cap_df.drop_duplicates(subset=['name'], keep=False)

In [10]:
clean_combined_cap_df.head(3)

,name,market_cap
0,Bitcoin,1206547874874
1,Ethereum,320467809737
2,Tether,115641341009


In [27]:
# Save as csv
clean_combined_cap_df.to_csv('clean_combined_cap_df_all.csv')

## Whitepaper DataFrame

Need to scrape extract information from whitepapers found online at https://whitepaper.io/

### Getting WP URLs

Firstly that means scraping the links to the online WP PDF.

This takes ages so is being done in a separate script.

Then we combine the saved csvs.

In [127]:
wp_url_df.columns = ['raw_wp_url', 'name']

In [129]:
wp_url_df[['name', 'raw_wp_url']]

,name,raw_wp_url
0,kava_swap,https://whitepaper.io/document/0/kava-swap-whi...
1,iqeon,https://whitepaper.io/document/0/iqeon-whitepaper
2,wagerr,https://whitepaper.io/document/160/wagerr-whit...
3,internet_node_token,https://whitepaper.io/document/211/internet-no...
4,cybervein,https://whitepaper.io/document/138/cybervein-w...
...,...,...
373,frogex,https://whitepaper.io/document/0/frogex-whitep...
374,dapp_token,https://whitepaper.io/document/0/dapp-token-wh...
375,whole_network,https://whitepaper.io/document/0/whole-network...
376,bankroll_network,https://whitepaper.io/document/0/bankroll-netw...


In [137]:
# Read csvs
csv_list = [
    'wp_url_df_50_100.csv',
    'wp_url_df_100_150.csv',
    'wp_url_df_150_200.csv',
    'wp_url_df_200_242.csv'
]
# First set as df
wp_url_df = pd.read_csv('wp_url_df_1_50.csv', encoding='utf-8', index_col=0)
# Read as df and union
for csv in csv_list:
    new_df = pd.read_csv(csv, encoding='utf-8', index_col=0)
    new_df.columns = ['raw_wp_url', 'name']  # correct column name fuck up
    wp_url_df = pd.concat([wp_url_df, new_df[['name', 'raw_wp_url']]])
# Reset index
wp_url_df = wp_url_df.drop_duplicates().reset_index()[['name', 'raw_wp_url']]
# Save to csv
wp_url_df.to_csv('wp_url_df_total.csv')

In [138]:
wp_url_df

,name,raw_wp_url
0,bitcoin,https://whitepaper.io/document/0/bitcoin-white...
1,ethereum,https://whitepaper.io/document/718/ethereum-wh...
2,tether,https://whitepaper.io/document/6/tether-whitep...
3,usd_coin,https://whitepaper.io/document/716/usd-coin-wh...
4,binance,https://whitepaper.io/document/10/binance-whit...
...,...,...
1402,velo,https://whitepaper.io/document/0/velo-whitepaper
1403,yuse,https://whitepaper.io/document/0/yuse-whitepaper
1404,hulk_inu,https://whitepaper.io/document/0/hulk-inu-whit...
1405,karbun,https://whitepaper.io/document/0/karbun-whitep...


### Get PDF URLs

Then we get the PDF link from those pages to pass to our PDF viewer,

In [140]:
# Function for getting pdf url from raw wp link
def get_pdf_url(raw_wp_url):
    result = requests.get(raw_wp_url, headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup.find('div', class_="flex flex-col flex-1").object.attrs['data']

In [141]:
# Use this function to add pdf url to our df
wp_url_df['pdf_url'] = wp_url_df['raw_wp_url'].apply(lambda x: get_pdf_url(x))

In [142]:
# Save as csv
wp_url_df.to_csv("pdf_url.csv")

### Get WP attributes

Use PyPDF2 to extract some useful attributes from these PDFs.

In [147]:
pdf_att_dict_list = []  # instantiate empty list
for url in wp_url_df['pdf_url']:
    try:
        coin_feature_dict = {}
        coin_feature_dict['pdf_url'] = url
        response = requests.get(url)
        
        with io.BytesIO(response.content) as f:
            pdf = PdfReader(f)
            
            coin_feature_dict['number_pages'] = len(pdf.pages)
            
            number_images = 0
            number_characters = 0
            number_equations = 0
        
            for page in pdf.pages:
                try:
                    number_images += len(page.images)
                except:
                    number_images += 1
                number_characters += len(page.extract_text())
                number_equations += page.extract_text().count("=")
    
            coin_feature_dict['number_images'] = number_images
            coin_feature_dict['number_characters'] = number_characters
            coin_feature_dict['number_equations'] = number_equations
    
        pdf_att_dict_list.append(coin_feature_dict)
    except:
        pass

incorrect startxref pointer(1)
unknown widths : 
[0, IndirectObject(196, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(191, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(186, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(181, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(176, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(171, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(166, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(161, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(156, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(151, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(196, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(191, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(186, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(181, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(176, 0, 2910450455120)]
unknown widths : 
[0, IndirectObject(171, 0, 291045045

In [148]:
# Make a dataframe with all pdf data
pdf_att_dict_df = pd.DataFrame(pdf_att_dict_list)

In [149]:
# Combine with url df
combined_wp_df = wp_url_df.merge(pdf_att_dict_df, how='inner', on='pdf_url')

In [150]:
combined_wp_df

,name,raw_wp_url,pdf_url,number_pages,number_images,number_characters,number_equations
0,bitcoin,https://whitepaper.io/document/0/bitcoin-white...,https://api-new.whitepaper.io/documents/pdf?id...,9,0,21807,80
1,ethereum,https://whitepaper.io/document/718/ethereum-wh...,https://api-new.whitepaper.io/documents/pdf?id...,42,6,88334,22
2,tether,https://whitepaper.io/document/6/tether-whitep...,https://api-new.whitepaper.io/documents/pdf?id...,20,4,39064,8
3,usd_coin,https://whitepaper.io/document/716/usd-coin-wh...,https://api-new.whitepaper.io/documents/pdf?id...,24,3,57036,0
4,binance,https://whitepaper.io/document/10/binance-whit...,https://api-new.whitepaper.io/documents/pdf?id...,17,35,21998,1
...,...,...,...,...,...,...,...
1263,velo,https://whitepaper.io/document/0/velo-whitepaper,https://api-new.whitepaper.io/documents/pdf?sl...,27,3,54064,5
1264,yuse,https://whitepaper.io/document/0/yuse-whitepaper,https://api-new.whitepaper.io/documents/pdf?sl...,35,14,35841,0
1265,hulk_inu,https://whitepaper.io/document/0/hulk-inu-whit...,https://api-new.whitepaper.io/documents/pdf?sl...,11,48,6053,0
1266,karbun,https://whitepaper.io/document/0/karbun-whitep...,https://api-new.whitepaper.io/documents/pdf?sl...,50,1006,50498,0


In [151]:
combined_wp_df.duplicated(subset='name').sum()

0

In [152]:
clean_pdf_att_df = combined_wp_df

In [153]:
# Save as csv
clean_pdf_att_df.to_csv('clean_pdf_att_df.csv', encoding='utf-8')

## Combined DataFrame

In [154]:
# Create lower case column for join
clean_combined_cap_df['name_lower'] = clean_combined_cap_df['name'].apply(lambda x: x.lower().replace(" ", "_"))

C:\Users\haydn\AppData\Local\Temp\ipykernel_18996\1195583360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_combined_cap_df['name_lower'] = clean_combined_cap_df['name'].apply(lambda x: x.lower().replace(" ", "_"))


In [159]:
clean_combined_cap_df

,name,market_cap,name_lower
0,Bitcoin,1206547874874,bitcoin
1,Ethereum,320467809737,ethereum
2,Tether,115641341009,tether
3,BNB,77731918080,bnb
4,Solana,72770697539,solana
...,...,...,...
2791,TradeStars,23768,tradestars
2792,Vesper V-Dollar,23614,vesper_v-dollar
2793,Insula,23494,insula
2794,Ducky City,23284,ducky_city


In [160]:
clean_pdf_att_df

,name,raw_wp_url,pdf_url,number_pages,number_images,number_characters,number_equations
0,bitcoin,https://whitepaper.io/document/0/bitcoin-white...,https://api-new.whitepaper.io/documents/pdf?id...,9,0,21807,80
1,ethereum,https://whitepaper.io/document/718/ethereum-wh...,https://api-new.whitepaper.io/documents/pdf?id...,42,6,88334,22
2,tether,https://whitepaper.io/document/6/tether-whitep...,https://api-new.whitepaper.io/documents/pdf?id...,20,4,39064,8
3,usd_coin,https://whitepaper.io/document/716/usd-coin-wh...,https://api-new.whitepaper.io/documents/pdf?id...,24,3,57036,0
4,binance,https://whitepaper.io/document/10/binance-whit...,https://api-new.whitepaper.io/documents/pdf?id...,17,35,21998,1
...,...,...,...,...,...,...,...
1263,velo,https://whitepaper.io/document/0/velo-whitepaper,https://api-new.whitepaper.io/documents/pdf?sl...,27,3,54064,5
1264,yuse,https://whitepaper.io/document/0/yuse-whitepaper,https://api-new.whitepaper.io/documents/pdf?sl...,35,14,35841,0
1265,hulk_inu,https://whitepaper.io/document/0/hulk-inu-whit...,https://api-new.whitepaper.io/documents/pdf?sl...,11,48,6053,0
1266,karbun,https://whitepaper.io/document/0/karbun-whitep...,https://api-new.whitepaper.io/documents/pdf?sl...,50,1006,50498,0


VERIFY THIS IS BIGGEST OVERLAP!

In [155]:
# Join on lower case name
combined_total_df = clean_combined_cap_df.merge(
    right=clean_pdf_att_df,
    how='inner',
    left_on='name_lower',
    right_on='name'
)

In [156]:
combined_total_df

,name_x,market_cap,name_lower,name_y,raw_wp_url,pdf_url,number_pages,number_images,number_characters,number_equations
0,Bitcoin,1206547874874,bitcoin,bitcoin,https://whitepaper.io/document/0/bitcoin-white...,https://api-new.whitepaper.io/documents/pdf?id...,9,0,21807,80
1,Ethereum,320467809737,ethereum,ethereum,https://whitepaper.io/document/718/ethereum-wh...,https://api-new.whitepaper.io/documents/pdf?id...,42,6,88334,22
2,Tether,115641341009,tether,tether,https://whitepaper.io/document/6/tether-whitep...,https://api-new.whitepaper.io/documents/pdf?id...,20,4,39064,8
3,Solana,72770697539,solana,solana,https://whitepaper.io/document/602/solana-whit...,https://api-new.whitepaper.io/documents/pdf?id...,32,13,46055,11
4,Dogecoin,15948639496,dogecoin,dogecoin,https://whitepaper.io/document/672/dogecoin-wh...,https://api-new.whitepaper.io/documents/pdf?id...,5,26,6855,30
...,...,...,...,...,...,...,...,...,...,...
328,Bismuth,54904,bismuth,bismuth,https://whitepaper.io/document/0/bismuth-white...,https://api-new.whitepaper.io/documents/pdf?sl...,32,32,56779,32
329,HappyFans,52795,happyfans,happyfans,https://whitepaper.io/document/0/happyfans-whi...,https://api-new.whitepaper.io/documents/pdf?sl...,11,0,0,0
330,Dogira,48031,dogira,dogira,https://whitepaper.io/document/0/dogira-whitep...,https://api-new.whitepaper.io/documents/pdf?sl...,36,0,0,0
331,Lunyr,35256,lunyr,lunyr,https://whitepaper.io/document/222/lunyr-white...,https://api-new.whitepaper.io/documents/pdf?id...,21,12,47406,0


In [161]:
# Reformat final DF and save
raw_df_to_save = combined_total_df[['name_x', 'market_cap', 'number_pages', 'number_images', 'number_characters', 'number_equations']]
raw_df_to_save.rename({"name_x": "name"}, inplace=True)
raw_df_to_save.to_csv(
    path_or_buf='raw_features.csv',
    encoding='utf-8'
)

C:\Users\haydn\AppData\Local\Temp\ipykernel_18996\2163949448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df_to_save.rename({"name_x": "name"}, inplace=True)
